In [ ]:
# @title 1. Setup and Imports
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# --- CONFIGURATION ---
# Path to the folder containing the 27 feature CSV files
# Ensure the path ends with a slash /
FEATURE_FOLDER_PATH = '/content/drive/MyDrive/Thesis data/Features/'

# Path to the Errors CSV
ERRORS_FILE_PATH = '/content/drive/MyDrive/Thesis data/Errors.csv'

# Path to save the final processed dataset
OUTPUT_PATH = '/content/drive/MyDrive/Thesis data/Processed_Dataset.csv'

# Correlation threshold for removing features (0.95 = 95% correlation)
CORR_THRESHOLD = 0.95
# ---------------------

print("Libraries loaded and Drive mounted.")

Mounted at /content/drive
Libraries loaded and Drive mounted.


In [ ]:
# @title 2. Load and Join Feature Data
def load_and_merge_features(folder_path):
    # Get all csv files in the folder
    all_files = glob.glob(folder_path + "*.csv")

    if len(all_files) == 0:
        raise ValueError(f"No CSV files found in {folder_path}. Check your path.")

    print(f"Found {len(all_files)} files. Beginning merge process...")

    df_list = []

    # Iterate through files with a progress bar
    for filename in tqdm(all_files, desc="Loading Files"):
        # Read CSV
        temp_df = pd.read_csv(filename)

        # Ensure Timestamp is datetime
        temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'])

        # Set index to Timestamp for easy joining
        temp_df.set_index('Timestamp', inplace=True)

        # Sort index to ensure time monotonicity
        temp_df.sort_index(inplace=True)

        # Remove duplicate indices if any exist within a file
        temp_df = temp_df[~temp_df.index.duplicated(keep='first')]

        df_list.append(temp_df)

    # Concatenate all dataframes along columns (axis=1)
    # This aligns them by the Timestamp index automatically
    full_df = pd.concat(df_list, axis=1)

    print(f"Merge complete. Raw shape: {full_df.shape}")
    return full_df

# Execute
df_main = load_and_merge_features(FEATURE_FOLDER_PATH)

Found 29 files. Beginning merge process...


Loading Files:   0%|          | 0/29 [00:00<?, ?it/s]

Merge complete. Raw shape: (1915106, 29)


In [ ]:
# Dictionary mapping Dutch names to English translations
column_translations = {
    'MT05 Pulsievocht Meting': 'MT05 Supply Humidity Measurement',
    'TV15 Stuurwaarde': 'TV15 Control Value',
    'MV06 Vochtregeling Stuurwaarde': 'MV06 Humidity Control Value',
    'RV Buitenvochtigheid': 'RV Outdoor Humidity',
    'TV13 Stuurwaarde': 'TV13 Control Value',
    'MV13 Vochtregeling Stuurwaarde': 'MV13 Humidity Control Value',
    'BP02 Stuurwaarde': 'BP02 Control Value',
    'MT08 Extractievocht Meting': 'MT08 Extraction Humidity Measurement',
    'PT13 Extractiedruk Meting': 'PT13 Extraction Pressure Measurement',
    'PT03 Extractiedruk Meting': 'PT03 Extraction Pressure Measurement',
    'PT12 Pulsiedruk Meting': 'PT12 Supply Pressure Measurement',
    'PT02 Pulsiedruk Meting': 'PT02 Supply Pressure Measurement',
    'MT04 Extractievocht Meting': 'MT04 Extraction Humidity Measurement',
    'MV02 Vochtregeling Stuurwaarde': 'MV02 Humidity Control Value',
    'TV04 Stuurwaarde': 'TV04 Control Value',
    'TT08 Buitentemperatuur Meting': 'TT08 Outdoor Temperature Measurement',
    'TT01 Pulsietemperatuur Meting': 'TT01 Supply Temperature Measurement',
    'MT01 Pulsievocht Meting': 'MT01 Supply Humidity Measurement',
    'TV02 Stuurwaarde': 'TV02 Control Value',
    'MV12 Vochtregeling Stuurwaarde': 'MV12 Humidity Control Value',
    'TT12 Pulsietemperatuur Meting': 'TT12 Supply Temperature Measurement',
    'MV10 Vochtregeling Stuurwaarde': 'MV10 Humidity Control Value',
    'BP01 Stuurwaarde': 'BP01 Control Value',
    'TV22 Stuurwaarde': 'TV22 Control Value',
    'TT19 Pulsietemperatuur Meting': 'TT19 Supply Temperature Measurement',
    'MT09 Pulsievocht Meting': 'MT09 Supply Humidity Measurement',
    'TV20 Stuurwaarde': 'TV20 Control Value',
    'BP03 Stuurwaarde': 'BP03 Control Value',
    'MV14 Vochtregeling Stuurwaarde': 'MV14 Humidity Control Control Value'
}

# Rename the columns in place
df_main.rename(columns=column_translations, inplace=True)

# Verify the changes
print(df_main.columns.tolist())

['MT05 Supply Humidity Measurement', 'TV15 Control Value', 'MV06 Humidity Control Value', 'RV Outdoor Humidity', 'TV13 Control Value', 'MV13 Humidity Control Value', 'BP02 Control Value', 'MT08 Extraction Humidity Measurement', 'PT13 Extraction Pressure Measurement', 'PT03 Extraction Pressure Measurement', 'PT12 Supply Pressure Measurement', 'PT02 Supply Pressure Measurement', 'MT04 Extraction Humidity Measurement', 'MV02 Humidity Control Value', 'TV04 Control Value', 'TT08 Outdoor Temperature Measurement', 'TT01 Supply Temperature Measurement', 'MT01 Supply Humidity Measurement', 'TV02 Control Value', 'MV12 Humidity Control Value', 'TT12 Supply Temperature Measurement', 'MV10 Humidity Control Value', 'BP01 Control Value', 'TV22 Control Value', 'TT19 Supply Temperature Measurement', 'MT09 Supply Humidity Measurement', 'TV20 Control Value', 'BP03 Control Value', 'MV14 Humidity Control Control Value']


In [ ]:
df_main.head()

,MT05 Supply Humidity Measurement,TV15 Control Value,MV06 Humidity Control Value,RV Outdoor Humidity,TV13 Control Value,MV13 Humidity Control Value,BP02 Control Value,MT08 Extraction Humidity Measurement,PT13 Extraction Pressure Measurement,PT03 Extraction Pressure Measurement,...,MV12 Humidity Control Value,TT12 Supply Temperature Measurement,MV10 Humidity Control Value,BP01 Control Value,TV22 Control Value,TT19 Supply Temperature Measurement,MT09 Supply Humidity Measurement,TV20 Control Value,BP03 Control Value,MV14 Humidity Control Control Value
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:01:00,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.8,NaN,NaN,NaN,15.9,NaN,NaN,NaN,NaN
2022-01-01 00:02:00,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-351.0,-348.0,...,NaN,NaN,NaN,NaN,11.0,15.7,NaN,NaN,NaN,NaN
2022-01-01 00:03:00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-346.0,-352.0,...,NaN,15.2,NaN,NaN,9.0,15.6,72.0,NaN,NaN,NaN
2022-01-01 00:04:00,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-349.0,-349.0,...,NaN,15.5,NaN,NaN,NaN,15.9,74.0,NaN,NaN,NaN
2022-01-01 00:05:00,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-355.0,...,NaN,16.2,NaN,NaN,15.0,16.3,NaN,NaN,NaN,NaN


In [ ]:
# @title 3. Resample to 15 Minutes
def resample_data(df):
    print("Resampling data to 15-minute intervals (Mean)...")

    # Resample using mean
    df_resampled = df.resample('15T').mean()

    # Check for NaNs created by resampling (gaps in data)
    nan_count = df_resampled.isna().sum().sum()
    if nan_count > 0:
        print(f"Warning: {nan_count} missing values found after resampling.")
        print("Imputing missing values using Forward Fill (ffill)...")
        df_resampled = df_resampled.ffill()
        # If any remain at the very start, backfill
        df_resampled = df_resampled.bfill()

    print(f"Resampling complete. New shape: {df_resampled.shape}")
    return df_resampled

df_resampled = resample_data(df_main)

Resampling data to 15-minute intervals (Mean)...


/tmp/ipython-input-904773821.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('15T').mean()


Imputing missing values using Forward Fill (ffill)...
Resampling complete. New shape: (128605, 29)


In [ ]:
# @title 4. Integrate Fault Labels
def label_faults(df, errors_path):
    print("Loading Error file...")
    df_errors = pd.read_csv(errors_path)

    # Convert timestamps to datetime
    df_errors['Timestamp Begin'] = pd.to_datetime(df_errors['Timestamp Begin'])
    df_errors['Timestamp End'] = pd.to_datetime(df_errors['Timestamp End'])

    # Calculate duration
    df_errors['duration'] = df_errors['Timestamp End'] - df_errors['Timestamp Begin']

    # Filter: Keep only faults <= 24 hours
    initial_count = len(df_errors)
    df_errors = df_errors[df_errors['duration'] <= pd.Timedelta(hours=24)]
    dropped_count = initial_count - len(df_errors)
    print(f"Dropped {dropped_count} faults lasting longer than 24 hours.")

    # Initialize Target Column (0 = Normal, 1 = Fault)
    df['Fault'] = 0

    print("Marking faults in the main dataframe...")
    # Iterate through valid errors and mark the range in the main df
    for index, row in tqdm(df_errors.iterrows(), total=df_errors.shape[0], desc="Labeling"):
        start_time = row['Timestamp Begin']
        end_time = row['Timestamp End']

        # Determine the label (assuming 'Fault' col in errors.csv helps,
        # but usually presence in this file implies a fault. We mark as 1).
        # If your Errors.csv 'Fault' column contains specific codes, you can use that.
        # Here we do binary classification: 1 for fault presence.

        # Using slicing to set values efficiently
        df.loc[start_time:end_time, 'Fault'] = 1

    print(f"Labeling complete. \nClass Balance:\n{df['Fault'].value_counts()}")
    return df

df_labeled = label_faults(df_resampled, ERRORS_FILE_PATH)

Loading Error file...
Dropped 24 faults lasting longer than 24 hours.
Marking faults in the main dataframe...


Labeling:   0%|          | 0/1607 [00:00<?, ?it/s]

Labeling complete. 
Class Balance:
Fault
0    126048
1      2557
Name: count, dtype: int64


In [ ]:
# @title 5. Feature Engineering (Rolling Windows)
def create_rolling_features(df):
    # Separate features and target
    target = df['Fault']
    features = df.drop(columns=['Fault'])

    # Define timeframes in terms of rows (since data is 15 mins)
    # 1 hour = 4 * 15 mins
    # 6 hours = 24 * 15 mins
    # 24 hours = 96 * 15 mins
    windows = {
        '1h': 4,
        '6h': 24,
        '24h': 96
    }

    # List to hold new feature dataframes
    new_features_list = [features]

    print("Generating rolling features...")

    # Loop through each window size
    for window_name, window_size in windows.items():
        print(f"Processing {window_name} window...")

        # Calculate Rolling Stats
        # mean, max, min, sum, var
        roll = features.rolling(window=window_size)

        feat_mean = roll.mean().add_suffix(f'_mean_{window_name}')
        feat_max = roll.max().add_suffix(f'_max_{window_name}')
        feat_min = roll.min().add_suffix(f'_min_{window_name}')
        feat_sum = roll.sum().add_suffix(f'_sum_{window_name}')
        feat_var = roll.var().add_suffix(f'_var_{window_name}')

        # Rate of Change (RoC)
        # Calculated as current value - value 'window_size' steps ago
        feat_roc = features.diff(periods=window_size).add_suffix(f'_roc_{window_name}')

        # Collect them
        new_features_list.extend([feat_mean, feat_max, feat_min, feat_sum, feat_var, feat_roc])

    # Concatenate all features
    df_engineered = pd.concat(new_features_list, axis=1)

    # Add target back
    df_engineered['Fault'] = target

    # Drop rows with NaNs created by the largest rolling window (start of dataset)
    original_len = len(df_engineered)
    df_engineered.dropna(inplace=True)
    print(f"Dropped {original_len - len(df_engineered)} rows due to rolling window initialization.")

    print(f"Feature Engineering complete. Total columns: {df_engineered.shape[1]}")
    return df_engineered

df_features = create_rolling_features(df_labeled)

Generating rolling features...
Processing 1h window...
Processing 6h window...
Processing 24h window...
Dropped 96 rows due to rolling window initialization.
Feature Engineering complete. Total columns: 552


In [ ]:
# @title 6. Correlation Filter
def remove_highly_correlated_features(df, threshold=0.95):
    print(f"Calculating correlation matrix (Threshold: {threshold})...")

    # Exclude Target from correlation check
    features_only = df.drop(columns=['Fault'])

    # Compute correlation matrix
    corr_matrix = features_only.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    print(f"Identified {len(to_drop)} features to drop due to high correlation.")

    # Drop features
    df_reduced = df.drop(columns=to_drop)

    print(f"Dimensionality reduction complete. Final shape: {df_reduced.shape}")
    return df_reduced

df_final = remove_highly_correlated_features(df_features, CORR_THRESHOLD)

Calculating correlation matrix (Threshold: 0.95)...
Identified 219 features to drop due to high correlation.
Dimensionality reduction complete. Final shape: (128509, 333)


In [ ]:
# @title 7. Save Dataset
def save_dataset(df, path):
    print(f"Saving data to {path}...")
    df.to_csv(path)
    print("Save complete.")

save_dataset(df_final, OUTPUT_PATH)

# Visual check of the first few rows
df_final.head()

Saving data to /content/drive/MyDrive/Thesis data/Processed_Dataset.csv...
Save complete.


,MT05 Supply Humidity Measurement,TV15 Control Value,MV06 Humidity Control Value,RV Outdoor Humidity,TV13 Control Value,MV13 Humidity Control Value,BP02 Control Value,MT08 Extraction Humidity Measurement,PT13 Extraction Pressure Measurement,PT12 Supply Pressure Measurement,...,TT12 Supply Temperature Measurement_roc_24h,MV10 Humidity Control Value_roc_24h,BP01 Control Value_roc_24h,TV22 Control Value_roc_24h,TT19 Supply Temperature Measurement_roc_24h,MT09 Supply Humidity Measurement_roc_24h,TV20 Control Value_roc_24h,BP03 Control Value_roc_24h,MV14 Humidity Control Control Value_roc_24h,Fault
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-02 00:00:00,61.500000,7.000000,27.000000,81.58276,0.0,0.0,65.0,39.0,-347.000000,399.750000,...,-0.086667,24.600000,0.0,1.233333,-0.031818,-10.600000,0.0,0.0,0.0,0
2022-01-02 00:15:00,62.000000,6.142857,26.600000,81.58276,0.0,0.0,64.0,40.0,-347.916667,399.900000,...,-0.150000,24.250000,-1.0,-0.642857,-0.081111,-10.416667,0.0,-1.0,0.0,0
2022-01-02 00:30:00,61.333333,6.750000,26.833333,81.58276,0.0,0.0,65.0,40.0,-347.888889,401.230769,...,-0.141818,24.166667,0.0,0.750000,-0.032857,-10.666667,0.0,0.0,0.0,0
2022-01-02 00:45:00,61.800000,8.166667,26.250000,81.58276,0.0,0.0,65.0,40.0,-347.500000,402.666667,...,0.065079,24.000000,0.0,0.885714,0.024444,-10.600000,0.0,0.0,0.0,0
2022-01-02 01:00:00,61.500000,6.333333,27.750000,81.58276,0.0,0.0,65.0,40.0,-348.285714,399.500000,...,0.055000,23.400000,0.0,0.791667,-0.022222,-10.666667,0.0,0.0,0.0,0
